<a href="https://colab.research.google.com/github/mesami8/ArchInternship/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Setup: Install Required Packages ---
!pip install -U unsloth bitsandbytes transformers faiss-cpu sentence-transformers
# Re-run this cell first
!pip install --upgrade --quiet unsloth bitsandbytes transformers faiss-cpu sentence-transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.8/299.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.6/504.6 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.7/166.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1

In [ ]:
# --- Imports ---
import torch
import faiss
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from IPython.display import display, Markdown
# from unsloth.hf_trainer import load_model # Removed as it's not available
from unsloth import FastLanguageModel # Use FastLanguageModel

In [ ]:
# --- Load Unsloth 4-bit Quantized Model ---

# from unsloth.hf_trainer import load_model # Removed as it's not available
from unsloth import FastLanguageModel # Use FastLanguageModel

# model, tokenizer = load_model( # Removed as it's not available
model, tokenizer = FastLanguageModel.from_pretrained( # Use FastLanguageModel
    model_name="unsloth/mistral-7b-bnb-4bit",  # You can change this to another 4bit model from Unsloth
    max_seq_length=4096,
    dtype=None,  # Let Unsloth decide
    load_in_4bit=True,
)

In [ ]:
# --- Load Embedding Model ---
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Small and fast


In [ ]:
# --- Your Domain-Specific Docs ---
documents = [
    "The mitochondria is the powerhouse of the cell.",
    "Transformers are models based on self-attention mechanisms.",
    "Python is a programming language used for many AI tasks.",
    "Unsloth is a library that enables loading 4-bit quantized models efficiently.",
    "Retrieval-Augmented Generation (RAG) combines search and generation."
]


In [ ]:
# --- Chunking Logic (optional if documents are small) ---
# You can add chunking if you're processing long docs.

# --- Embed Documents ---
doc_embeddings = embedding_model.encode(documents, convert_to_numpy=True)

In [ ]:
# --- FAISS Indexing ---
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

In [ ]:
# --- Mapping Index to Document Chunks ---
index_to_doc = {i: doc for i, doc in enumerate(documents)}

In [ ]:

# --- RAG Function ---
def rag_query(query, top_k=2):
    # Step 1: Embed the query
    query_embedding = embedding_model.encode([query])

    # Step 2: Retrieve top-k similar docs
    D, I = index.search(np.array(query_embedding), top_k)

    # Step 3: Get relevant context
    retrieved_docs = [index_to_doc[i] for i in I[0]]
    context = "\n".join(retrieved_docs)

    # Step 4: Build Prompt
    prompt = f"""Answer the question using the information below:\n\n{context}\n\nQuestion: {query}\nAnswer:"""

    # Step 5: Generate Answer
    output = text_gen(prompt, max_new_tokens=100, do_sample=True, temperature=0.7)[0]["generated_text"]

    # Only return the answer portion
    answer = output.split("Answer:")[-1].strip()

    return answer


In [ ]:
# --- Example Query ---
query = "The mitochondria is the powerhouse of the cell?"
answer = rag_query(query)
display(Markdown(f"**Q:** {query}\n\n**A:** {answer}"))

In [ ]:
# --- Create Text Generation Pipeline ---
text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
)